In [1]:
import sys
import os
import numpy as np
import time
from imp import reload

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
import testing.inputs  
reload(testing.inputs)
from testing.inputs import *
from L4 import DisplayWindows

new directory is: C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE


C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py


## Step 1. Load the CE System with a Config File

Select one of the predesigned configurations (TE300, Eclipse Ti, Simulation). See an example config under "/config/test-system.cfg". 

Change the variable CONFIG_PATH to the absolute path where the config file resides. 

Be sure to make the string a raw string by prepending *'r'*. **Example:** r"C:\path\to\config\"*

In [3]:
CONFIG_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\TE300.cfg"
from L3 import SystemsBuilder
reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()
try:
    ce_system.load_config(CONFIG_PATH)
    ce_system.open_controllers()
    ce_system.startup_utilities()
except Exception as e:
    
    print("Error Opening CE System")
    print(e)
    ce_system.close_controllers()

{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x00000230BD0BD548>, 'xy_stage': <L2.XYControl.PriorXY object at 0x00000230BDC0EA88>, 'objective': <L2.ZControl.PriorZ object at 0x00000230BD0BD408>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x00000230BD0BD508>, 'inlet_rgb': <L2.LightControl.RGBArduino object at 0x00000230BD0BD448>, 'inlet_z': <L2.ZControl.KinesisZ object at 0x00000230BD0BD108>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x00000230BD0BD4C8>, 'camera': <L2.CameraControl.PycromanagerControl object at 0x00000230BD0BD808>, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x00000230BD0BD088>}


# Step 2. Prepare the Capillary for Electrophoresis

We need to prep the capillary for CE. This involves rinsing the capillary surface and removing contaminants as well as re-establsihing a deprotonated silanol layer. We also need to align the capillary with detection objective. 

### Step 2.1 Rinse the capillary (Repeatable)

Rinse the capillary according to your method. A good place to start is 10 min with HCl, 30 min with NaOH, 2 min with water. If this is your first rinse of the day,rinse with an empty vial first to make sure there are no leaks on the outlet side of the apparatus. 

In [16]:
# Start Rinse
ce_system.outlet_pressure.rinse_pressure()

In [15]:
# Stop Rinse 
ce_system.outlet_pressure.release()

### Step 2.2 Align the capillary with fluorescein. 

Place a vial with 125 nM fluorescein in the outlet. Once the pressure is rinsing adjust the capillary position using the two set screws as well as the adjustable objective collar until the signal is maximized for the detector being used. (PMT gain voltage should be set to 250 mV, or the lowest setting with appreciable gain). 

In [17]:
# Run the CE View
from L4 import DisplayWindows
reload(DisplayWindows)
%matplotlib qt
ce_disp = DisplayWindows.PLTCEDisplay(ce_system)
ce_disp.show()
ce_system.detector.start()
ce_disp.start_live_view()

In [10]:
resp = get_yes_no("Is fluorescein in the outlet and ready to be rinsed with?")

if resp:
    ce_system.outlet_pressure.rinse_pressure()
    print("Adjust the capillary inlet while maximizing the signal")
if not resp:
    print("Please insert the fluorscein in the outlet, then re-run this cell.")

Is fluorescein in the outlet and ready to be rinsed with? y
Adjust the capillary inlet while maximizing the signal


When finished. Record the maximum signal, along with the detector used, its gain setting (or bias when using SiPM), and the fluorescein concentration in the calibration log (located on the desktop). 

Run the next cell to stop the pressure rinse. 

In [18]:
ce_system.outlet_pressure.release()

### Step 2.2 Post Fluorescein Rinse

Take a vial and add close to 4 mL of H20. Insert it into the outlet and then remove it. This removes any residual Fluorescein from the capillary, electrode, or other surfaces.  

Repeat with a second vial, with liquid slightly higher or the same height. 

Finally switch with separation buffer and rinse for 10 min. 

In [22]:
resp = get_yes_no("Have you done the pre-fluorescein rinses?")
if not resp:
    print("Please follow instructions in step 2.2")
else:
    resp = get_yes_no("Is the separation buffer placed in the outlet?")
    if resp:
        ce_system.outlet_pressure.rinse_pressure()
        print("Run the next cell when finished")
    else:
        print("Please place the separation buffer in the outlet before rinsing.")

Have you done the pre-fluorescein rinses? y
Is the separation buffer placed in the outlet? y
Run the next cell when finished


In [25]:
ce_system.outlet_pressure.release()

# Step 3. Calibrate the Capillary Inlet

The method for calibration depends on the z-stage used to raise the capillary inlet. Select the correct route for the hardware employed. 

## Thorlabs Kinesis Z Stage

### Step 3.1 Lower the Capillary (thorlabs)
Move the capillary to a safe location where it will not be crushed if the stage lowers too far. Using the controls on the Z-stage itself, lower the capillary till as far as it will go. 


In [100]:
resp = get_yes_no("Have you lowered the Kinesis Z stage to its lowest position? (y/n):  ")
if not resp:
    print("Please make sure the Kinesis stage is lowered to its lowest position before continuing")
else:
    print("Proceed to the next cell.")
    

Have you lowered the Kinesis Z stage to its lowest position? (y/n):   y
Proceed to the next cell.


### Step 3.2 Home the Capillary (thorlabs)

Thorlabs needs to register the home position. In the next cell we will call the go_home command to home the capillary. 

In [101]:
ce_system.inlet_z.homing()

### Step 3.1 (Arduino/Optics-Focus)
The optics focus moves the capillary all the way up during the home step. Once this is done, move the capillary to a safe position and lower (with 1-5mm increments) until the capillary is at 0 mm. 

In [12]:
ce_system.inlet_z.set_rel_z(-5)

In [ ]:
ce_system.inlet_z.set_z(0)

In [ ]:
resp = get_yes_no("Have you lowered the Arduino Z stage to 0 mm? (y/n):  ")
if not resp:
    print("Please make sure the Aduino stage is lowered to 1 mm position before continuing")
else:
    print("Proceed to the step 3.3 ")

### Step 3.3 Adjust Capillary Position (all stages)

Adjust the capillary position by sliding the detection apparatus up or down the optical support post. The capillary should be level with the bottom of the template for single-cell assays, or the top-of the template for well-based assays. 

*Aligning to the top of the template is a minor safety feature to prevent the user from sending the capillary to Zero position unexpetedly and crashing the capillary against the bottom of the template. However, capillary needs to be able to reach at least the glass slide of the cell reservoir for single-cell assays*


In [102]:
resp = get_yes_no("Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):  ")
if not resp:
    print("Please make sure the capillary is moved to the correct height with respect to the template before continuing")
else:
    print("Proceed to the next cell")

Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):   y
Proceed to the next cell


### Step 3.4 Raise the capillary to a safe height

The next cell will raise the capillary height to a location where it ***should*** be safe for any XY movements. Verify the capillary will not crash at this new height. 

In [103]:
target_z = get_float("Please enter a safe height to move the stage to: ")
# Move the capillary up and wait for it to stop
ce_system.inlet_z.set_z(target_z)
ce_system.inlet_z.wait_for_target(target_z)

resp = get_yes_no("Is the capillary at a safe location?: ")
if resp:
    print("Continue to next step")
else:
    print("Please raise the capillary to an appropriate height")

Please enter a safe height to move the stage to:  34
Is the capillary at a safe location?:  y
Continue to next step


# Step 4. Home the XY Stage

This next step provides the calibration for the XY axis for the XY stage and capillary inlet. 


### Step 4.1 (TE300, Prior)

The TE300 does not have the encoders wired to tell it where absolute 0,0 is. Thus, we need to tell the microcontroller where 0,0 is for each template. The home position will likely vary as you switch templates and you may need to investigate the template.txt file for the template you plan to use. 

1. Move the template's home well (0,0) over the objective using the motorized XY stage
2. Align the capillary over the well or the objective


In [111]:
resp = get_yes_no("Is the capillary aligned with the objective and home well?")
if resp:
    ce_system.xy_stage.set_home()
    print("Home set at this position.")
    print("Continue to next step")
else:
    print("Please raise align the capillary with the objective")   


Is the capillary aligned with the objective and home well? y


IndexError: list index out of range

In [119]:
f1 = ce_system.xy_stage.controller.send_command("Z \r")
f2 = ce_system.xy_stage.controller.send_command("\r")


In [130]:
f2 = ce_system.xy_stage.controller.send_command("Z \r")
f2

'0'

In [ ]:
response = self.controller.send_command("\r").split(',')


### Step 4.1 (Nikon Eclipse)

The nikon eclipse has a home position defined to the center of the template. It is best to use this as the home position. For the capillary alignment, align the capillary concentrically with the objective. The capillary should be within 0.5 mm of the center of the objective if using PCR tubes at any point. 

In [31]:
resp = get_yes_no("Is the capillary aligned with the objective?")
if resp:
    print("Continue to next step")
else:
    print("Please raise align the capillary with the objective")

Is the capillary aligned with the objective? y
Continue to next step


# Step 5 Verify your calibration

Steps 3-4 calibrate the capillaries position with the microscope's position. Now we need to verify that our alignment is correct. To do this we will move the capillary to a well and make sure that it is centered over the well. Then we will lower the capillary to a height specified by the method to make sure the capillary can reach the liquid. 


### Step 5.1. Load the Template File

The template file tells the program where to move the XY stage for each well as well as how high to raise the capillary inlet during a move. Change the variable TEMPLATE_PATH to the absolute path where the template.txt file resides. 

In [36]:
TEMPLATE_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\80WellTemplate.txt"
from L4 import AutomatedControl
from L4 import Trajectory
reload(AutomatedControl)
reload(Trajectory)


auto_program = AutomatedControl.AutoRun(ce_system)
auto_program.set_template(TEMPLATE_PATH)

### Step 5.2 Raise the capillary to a safe height 

In [23]:
z = get_float("Set the capillary to a safe travel height: ")
ce_system.inlet_z.set_z(z)
print("Proceed to next cell")

Set the capillary to a safe travel height:  25
Proceed to next cell


### Step 5.2 Center the Capillary over a well

In [27]:
name = get_value("Please enter the well name to move to (press enter to see a list): ", auto_program.template.wells.keys())
well = auto_program.template.wells[name]
ce_system.xy_stage.set_xy(well.xy)
resp = input("Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.")


Please enter the well name to move to (press enter to see a list): w_a0
Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.


In [28]:
ce_system.inlet_rgb.turn_off_channel('R')

'R:Off, G:Off, B:Off'

In [29]:
ce_system.inlet_rgb.turn_on_channel('B')

'R:Off, G:Off, B:On'

### Step 5.3 Lower the capillary to the height described in the method file


In [30]:
height_z = get_float("What height should the capillary be lowered to?")
prev_height = ce_system.inlet_z.read_z()
ce_system.inlet_z.set_z(height_z)
resp = get_yes_no("Is the capillary touching the liquid in this well? ")
if not resp:
    print("Adjust the height of the capillary manually (and carefully!) so that the capillary is touching the liquid")
else:
    print("Calibration Finished")
ce_system.inlet_z.set_z(prev_height)

What height should the capillary be lowered to? 5
Is the capillary touching the liquid in this well?  y
Calibration Finished


Optional> 
Read out the height you are currently at...

In [24]:
print("You are at {} mm".format(ce_system.inlet_z.read_z()))


You are at 9.277067 mm


In [13]:
resp = get_yes_no("Would you like to lower the capillary?")
if resp:
    resp = get_float("What would you like to lower it to?")
    ce_system.inlet_z.set_z(resp)
else:
    pass

Would you like to lower the capillary? y
What would you like to lower it to? 25


# Step 6. Load a method

Load a method file. Verify your method file in excel and make sure the run is set up as intended. You can add multiple files to the same auto_program


In [95]:
auto_program.methods=[]

In [96]:
file_to_add = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\CollectionDNA.txt"
auto_program.add_method(file_to_add)

In [97]:
auto_program.repetitions=6

If you need to reset the methods (remove and start over) run the next cell

### Step 6.2  Run your method (repeatable)

When you are ready to start your method, run the next cell. 

In [98]:
%matplotlib qt

ce_gram = DisplayWindows.PLTCEDisplay(ce_system)
#ce_disp.stop()
#ce_disp.show()
ce_gram.start_live_view()


In [99]:
auto_program.start_run()

C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\L4\DisplayWindows.py:270: UserWarning: Attempting to set identical left == right == 0.0 results in singular transformations; automatically expanding.
  ax1.set_xlim(min(data['time_data']), max(data['time_data']))
ERROR:root:Nidaq did not read samples correctly


In [82]:
auto_program.stop_run()

ERROR:root:ERR: <class 'L2.LaserControl.LaserAbstraction'> did not stop. 
 type object 'LaserAbstraction' has no attribute 'stop'


In [94]:
auto_program.traced_thread.is_alive()

False

In [92]:
while not auto_program._queue.empty():
    auto_program._queue.get()

In [91]:
ce_system.inlet_z.read_z()

1.0

### 6.3 Manual Single Cell Methods

For a manual single cell run, the method assumes the user will start the program after the cell as been loaded. The first step of the method should be the first step you want to perform following lysing and loading a cell into the capillary (ie. separation step). The method should end at the well where the cells are plated, or wherever you would like to regain manual control over the instrument. 

The injection/loading of the cell can be done in two ways. In either case at the start of the next cell are a few parameters for you to specify: **injection time, delay, gravity drop, and voltage level**. 
1. **Injection time** refers to how long the injection should last and is only applied in methods 6.3a. 
1. **Delay** is the amount of time to delay the injection steps after running the cell, the word 'Fire' will be printed once the delay is finished.
1. **Gravity drop** is how far to lower the outlet in mm from its current position. 
1. **Voltage level** is the amount of voltage to apply. 

In the first way the cell is loaded using a preset amount of time. The injection volume will be much more consistent, but if the cell is not loaded, the user will either choose to run a blank (no cell), rinse the capillary to remove the ECB, or start the auto-method. 

The second way, the loading parameters are held till the user presses the space-bar to continue the run. The injection volume may change run to run, but the user decides when to stop which means a higher likely-hood over loading a cell (at the expense of possible greater de-stacking. 





In [7]:
# General Injection Parameters
injection_time = 5 # in seconds
gravity_drop = 0 # in millimeters
voltage_level = 3 # in kilovolts
delay = 1 # inseconds

In [47]:

ce_disp = DisplayWindows.PLTMicroscopeDisplay(ce_system)
%matplotlib qt
ce_system.camera.stop()
ce_disp.show()
ce_disp.live_image()

In [182]:
ce_system.inlet_z.set_z(24)

In [25]:
ce_disp.percentiles=2
ce_disp.gain =1
ce_disp.gamma = 1.02

In [24]:
ce_disp.percentiles=1

In [13]:
ce_system.camera.set_exposure(5)

True

In [238]:
ce_system.camera.stop()

True

In [183]:
ce_system.outlet_pressure.rinse_pressure()


In [184]:
ce_system.outlet_pressure.release()

In [ ]:
ce_system.objective.

In [7]:
# Move the objective to the height and run this cell to record it for next time
obj_height = ce_system.objective.read_z()
ce_system.objective.max_z=1000000
ce_system.objective.set_rel_z(+.05)


In [29]:
# Lower the capillary 
ce_system.inlet_z.set_z(5)


In [115]:
ce_system.objective.read_z()

-0.0269

In [114]:
ce_system.objective._scale = 10000

In [112]:
ce_system.objective._offset =0

In [25]:
ce_system.inlet_z.set_rel_z(-.01)

In [26]:
# and record this height for next time 
cap_height = ce_system.inlet_z.read_z()

In [27]:
cap_difference = obj_height - cap_height

In [28]:
def get_cap_height(obj_height, cap_difference):
    cap_height = obj_height - cap_difference
    return cap_height

In [86]:
ce_system.objective.max_z = 100000

# Macros


In [49]:
# on reruns, you just need to run this command when you have the cells in focus
resp = get_yes_no("are the cells in focus? (You may crush the capillary if not...)")
if resp:
    obj_height = ce_system.objective.read_z()
    cap_z = get_cap_height(obj_height, cap_difference)
    ce_system.inlet_z.set_z(cap_z)
else:
    print("Bring cells into focus.")

are the cells in focus? (You may crush the capillary if not...) y


In [103]:
obj_height

273.1

In [35]:
ce_system.objective.set_rel_z(+0.035)

In [36]:
ce_system.inlet_z.set_rel_z(-0.04)

In [37]:
# General Injection Parameters
injection_time = 5 # in seconds
gravity_drop = 0 # in millimeters
voltage_level = 10 # in kilovolts
delay = 1 # inseconds

In [38]:
import threading


def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.high_voltage.start()
    start_time = time.time()
    while time.time()-start_time < injection_time:
        time.sleep(0.01)
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)
    

threading.Thread(target=limited_fire, args=(ce_system, gravity_drop, voltage_level, injection_time)).start()

Fire
Finished 5.002622365951538


In [39]:
ce_system.inlet_z.set_z(24)

In [277]:
ce_system.outlet_pressure.rinse_pressure()

In [276]:
ce_system.outlet_pressure.release()

In [44]:
auto_program.start_run()

In [261]:
auto_program.stop_run()

ERROR:root:ERR: <class 'L2.LaserControl.LaserAbstraction'> did not stop. 
 type object 'LaserAbstraction' has no attribute 'stop'


In [270]:
auto_program.traced_thread.isAlive()

True

In [271]:
auto_program.traced_thread.killed

True

In [37]:
import matplotlib.pyplot as plt 
plt.imshow(img, cmap='gray')

NameError: name 'img' is not defined

In [14]:
img = ce_system.camera.snap()

In [ ]:
ce_system

### 6.3a Injection Time Limited Loading

Run the following block when the cell is ready to by lysed and loaded. 

**For the eclipse system you will have to manually press the fire button after running the cell

### 6.3b Unlimited Loading

Run the following block when the cell is ready to by lysed and loaded. 

Run the block after when the loading process is complete. 

**For the eclipse system you will have to manually press the fire button after running the cell**


In [308]:
time.sleep(delay)
print('Fire')
ce_system.outlet_z.set_rel_z(-gravity_drop)
ce_system.high_voltage.set_voltage(voltage_level)
ce_system.high_voltage.start()
#ce_system.lysis_laser.laser_standby()
#ce_system.lysis_laser.laser_fire()
start_time = time.time()

Fire


In [310]:
ce_system.high_voltage.stop()
ce_system.outlet_z.set_rel_z(gravity_drop)

In [128]:
ce_system.objective.set_rel_z(0.045)

# Step 7. End of Day

1. Repeat Step 2.1, this time rinse for 5 min with water, then 30 min with air. Run the next cell when you are finished rinsing.
2. Turn off all equipment. 

In [17]:
ce_system.shutdown_utilities()

AttributeError: 'NoneType' object has no attribute 'shutdown'

In [ ]:
ce_system.close_controllers()

In [ ]:
ce_system.open_controllers()

In [ ]:
cam_disp = DisplayWindows.PLTMicroscopeDisplay(ce_system)

# Appendix A. Adding a Template


In [18]:
from L4.AutomatedControl import TemplateMaker
tm = TemplateMaker()

In [ ]:
Add a well 

In [ ]:
cam_disp.show()

In [ ]:
cam_disp.stop_live_image()

In [ ]:
cam_disp.percentiles = 0

In [ ]:
ce_system.camera.set_exposure(45)

In [ ]:
cam_disp.live_image()

In [278]:
ce_system.shutdown_utilities()

In [279]:
ce_system.inlet_rgb.turn_off_channel('R')

'R:Off, G:Off, B:Off'

In [282]:
ce_system.inlet_rgb.controller.send_command("LB0\n")

'LB0\r\n'

In [209]:
data = {'start':[], 'stop':[], 'diff':[], 'xy':[], 'start_img':[], 'stop_img':[]}

In [269]:
start_z = ce_system.objective.read_z()
ce_system.objective.set_rel_z(np.mean(data['diff'][-5:]))
ce_system.camera.stop()
start_img = ce_system.camera.snap()
ce_system.camera.continuous_snap()
np.mean(data['diff'][-5:])

0.03234000048191774

In [270]:
import numpy as np
stop_z = ce_system.objective.read_z()
ce_system.camera.stop()
stop_img = ce_system.camera.snap()
ce_system.camera.continuous_snap()
data['start'].append(start_z)
data['stop'].append(stop_z)
data['diff'].append(stop_z-start_z)
data['xy'].append(ce_system.xy_stage.read_xy())
data['start_img'].append(start_img)
data['stop_img'].append(stop_img)

print(f"Average : {np.mean(data['diff'])}")


Average : 0.03362857192968046


In [230]:
for key in data:
    data[key]=data[key][0:-1]

In [271]:
import pandas as pd
df = pd.DataFrame.from_dict(data)

C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\skimage\exposure\exposure.py:351: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)


In [329]:
%matplotlib qt
#auto_program.start_run()
ce_gram = DisplayWindows.PLTCEDisplay(ce_system)

#ce_gram.show()
ce_gram.start_live_view()

In [272]:
df.to_csv('lysis_data')